In [63]:
%matplotlib inline

from pathlib import Path
from collections import Counter, OrderedDict

import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist

import tensorflow as tf
    

import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import seaborn as sns

from nltk.corpus import stopwords
import nltk
from nltk.stem.porter import PorterStemmer


from collections import OrderedDict
from pathlib import Path

import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import seaborn as sns

import pyLDAvis
from pyLDAvis.sklearn import prepare

from wordcloud import WordCloud
from termcolor import colored

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import train_test_split
import joblib

import re

from random import randint



In [64]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Reshape, Dot, Embedding
from tensorflow.keras.preprocessing.sequence import skipgrams, make_sampling_table
from tensorflow.keras.callbacks import Callback, TensorBoard

from sklearn.decomposition import TruncatedSVD

In [65]:
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
if gpu_devices:
    print('Using GPU')
    tf.config.experimental.set_memory_growth(gpu_devices[0], True)
else:
    print('Using CPU')
    

Using GPU


In [66]:
results_path = Path('results')
model_path = Path('results', 'bbc')
if not model_path.exists():
    model_path.mkdir(exist_ok=True, parents=True)

In [67]:
path =  Path('./data/bbc')
files = sorted(list(path.glob('**/*.txt')))
doc_list = []
for i, file in enumerate(files):
    with open(str(file), encoding='latin1') as f:
        topic = file.parts[-2]
        lines = f.readlines()
        heading = lines[0].strip()
        body = ' '.join([l.strip() for l in lines[1:]])
        doc_list.append([topic.capitalize(), heading, body])
        


In [57]:
n_components = 5
topic_labels = ['Topic {}'.format(i) for i in range(1, n_components + 1)]

max_df = .2
min_df = 3
max_features = 2000

# used by sklearn: https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/feature_extraction/stop_words.py
stop_words = pd.read_csv('http://ir.dcs.gla.ac.uk/resources/linguistic_utils/stop_words', 
                         header=None, 
                         squeeze=True).tolist()


C:\Users\JBarr\AppData\Local\Temp\ipykernel_129156\1404930559.py:9: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  stop_words = pd.read_csv('http://ir.dcs.gla.ac.uk/resources/linguistic_utils/stop_words',


In [58]:



docs = pd.DataFrame(doc_list, columns=['Category', 'Heading', 'Article'])


train_docs, test_docs = train_test_split(docs,
                                         stratify=docs.Category,
                                         test_size= .5,
                                         random_state=42)



countvectorizer = CountVectorizer(max_df=max_df, 
                             min_df=min_df, 
                             stop_words=stop_words, 
                             max_features=max_features)

train_dtm = countvectorizer.fit_transform(train_docs.Article)
test_dtm = countvectorizer.transform(test_docs.Article)


lda_base = LatentDirichletAllocation(n_components=n_components,
                                     n_jobs=-1,
                                     learning_method='batch',
                                     max_iter=10)
lda_base.fit(train_dtm)

joblib.dump(lda_base, model_path / 'lda_10_iter.pkl')

['results\\bbc\\lda_10_iter.pkl']

In [73]:
import pickle
import os
model_path = Path('movieclassifier', 'pkl_objects')
if not model_path.exists():
    model_path.mkdir(exist_ok=True, parents=True)


In [74]:
lda_base = joblib.load(model_path / 'lda_10_iter.pkl') 


In [76]:
objPath = Path('pkl_objects')
classifier_base = joblib.load(objPath/'classifier.pkl') 
classifier_base

AttributeError: Can't get attribute 'tokenizer' on <module '__main__'>

In [32]:

Tfidfvectorizer = TfidfVectorizer(max_df=.25,
                             min_df=.01,
                             stop_words='english',
                             binary=False)
train_dtmTfidfv = Tfidfvectorizer.fit_transform(train_docs.Article)

test_dtmTfidfv = Tfidfvectorizer.transform(test_docs.Article)
train_token_countTfidfv = train_dtmTfidfv.sum(0).A.squeeze()
Tfidfvtokens = Tfidfvectorizer.get_feature_names()
Tfidfvword_count = pd.Series(train_token_countTfidfv,
                       index=Tfidfvtokens).sort_values(ascending=False)

svd = TruncatedSVD(n_components=n_components, n_iter=5, random_state=42)
svd.fit(train_dtmTfidfv)
  
train_doc_topicsTfidfv = svd.transform(train_dtmTfidfv)

train_resultTfidfv = pd.DataFrame(data=train_doc_topicsTfidfv,
                            columns=topic_labels,
                            index=train_docs.Category)


topics = pd.DataFrame(svd.components_.T, index=Tfidfvtokens, columns=topic_labels)


c:\Users\JBarr\anaconda3\envs\blockchainDev\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [33]:
test_evalTfidfv = pd.DataFrame(data=svd.transform(test_dtmTfidfv),
                         columns=topic_labels,
                         index=test_docs.Category)

In [34]:
test_evalTfidfv

,Topic 1,Topic 2,Topic 3,Topic 4,Topic 5
Category,,,,,
Tech,0.166735,0.011555,-0.089148,-0.010527,-0.129305
Sport,0.247760,-0.287505,0.159446,-0.167504,0.010546
Business,0.151718,0.073303,-0.030046,-0.040677,0.100706
Tech,0.136796,0.048634,-0.166585,-0.048980,-0.241851
Politics,0.142048,0.073519,0.070498,0.013955,-0.036789
...,...,...,...,...,...
Sport,0.167124,-0.135201,0.047897,-0.099982,-0.026431
Entertainment,0.162986,0.005018,-0.000710,0.039488,0.012043
Business,0.168069,0.087723,-0.086423,-0.037452,0.077776


In [21]:
'''


display(Tfidfvword_count.head(10), topics.loc[Tfidfvword_count.head(10).index],
)




top_words, top_vals = pd.DataFrame(), pd.DataFrame()
for topic, words_ in topics.items():
    top10 = words_.abs().nlargest(10).index
    vals = words_.loc[top10].values
    top_vals[topic] = vals
    top_words[topic] = top10.tolist()
sns.heatmap(pd.DataFrame(top_vals),
            annot=top_words,
            fmt='',
            center=0,
            cmap=sns.diverging_palette(0, 255, sep=1, n=256),
            ax=ax)
ax.set_title('Top Words per Topic')
sns.despine()
fig.tight_layout()

train_resultTfidfv.groupby(level='Category').mean().plot.bar(figsize=(14, 5), rot=0)

sns.set(font_scale=1.3)
resultTfidfv = pd.melt(test_evalTfidfv.assign(Data='Train').append(
    test_evalTfidfv.assign(Data='Test')).reset_index(),
                 id_vars=['Data', 'Category'],
                 var_name='Topic',
                 value_name='Weight')

g = sns.catplot(x='Category',
                y='Weight',
                hue='Topic',
                row='Data',
                kind='bar',
                data=resultTfidfv,
                aspect=3.5); 
'''

"\n\n\ndisplay(Tfidfvword_count.head(10), topics.loc[Tfidfvword_count.head(10).index],\n)\n\n\n\n\ntop_words, top_vals = pd.DataFrame(), pd.DataFrame()\nfor topic, words_ in topics.items():\n    top10 = words_.abs().nlargest(10).index\n    vals = words_.loc[top10].values\n    top_vals[topic] = vals\n    top_words[topic] = top10.tolist()\nsns.heatmap(pd.DataFrame(top_vals),\n            annot=top_words,\n            fmt='',\n            center=0,\n            cmap=sns.diverging_palette(0, 255, sep=1, n=256),\n            ax=ax)\nax.set_title('Top Words per Topic')\nsns.despine()\nfig.tight_layout()\n\ntrain_resultTfidfv.groupby(level='Category').mean().plot.bar(figsize=(14, 5), rot=0)\n\nsns.set(font_scale=1.3)\nresultTfidfv = pd.melt(test_evalTfidfv.assign(Data='Train').append(\n    test_evalTfidfv.assign(Data='Test')).reset_index(),\n                 id_vars=['Data', 'Category'],\n                 var_name='Topic',\n                 value_name='Weight')\n\ng = sns.catplot(x='Category'

In [62]:

import pickle
import os

dest = os.path.join('movieclassifier', 'pkl_objects')
if not os.path.exists(dest):
    os.makedirs(dest)

classifier_base = joblib.load('classifier.pkl') 
classifier_base

FileNotFoundError: [Errno 2] No such file or directory: 'classifier.pkl'